## Import des librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from pathlib import Path
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
import time

2024-07-03 08:09:59.693733: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-03 08:09:59.730877: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 08:10:00.631509: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Configurer TensorFlow pour utiliser le GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


2024-07-03 08:10:01.925295: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-03 08:10:01.958158: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-03 08:10:01.962662: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
K.clear_session()

## Configuration et Prétraitement des Données

In [4]:
# Paramètres
IMG_SIZE = (224, 224)
BATCH_SIZE = 4  # Réduction de la taille du batch
EPOCHS = 30
CWD = Path.cwd()
NEW_TRAIN = CWD / "sorted_data" / "train"
NEW_VAL = CWD / "sorted_data" / "val"
NEW_TEST = CWD / "sorted_data" / "test"
class_names = {0: 'NORMAL', 1: 'VIRUS', 2: 'BACTERIA'}
K = 5  # Nombre de folds pour la validation croisée

In [5]:
# Prétraitement des données
def preprocess_dataset(dataset):
    return dataset.map(lambda x, y: (preprocess_input(x), y))

In [6]:
def create_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten(name='new_flatten')(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    classifieur = Dense(3, activation='softmax')(x)

    new_model = Model(inputs=base_model.input, outputs=classifieur)
    return new_model, base_model

In [7]:
# Charger les données en utilisant des générateurs et les stocker temporairement sur le disque
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
data_flow = data_generator.flow_from_directory(NEW_TRAIN, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)

# Stocker les données sous forme de fichiers TFRecord temporaires pour éviter de surcharger la mémoire
tfrecord_dir = CWD / "tfrecord_temp"
tfrecord_dir.mkdir(parents=True, exist_ok=True)

Found 12485 images belonging to 3 classes.


In [8]:
def image_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_example(image, label):
    feature = {
        'image': image_feature(image),
        'label': bytes_feature(tf.io.serialize_tensor(label).numpy())
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


In [9]:
def write_tfrecord(data_flow, tfrecord_dir, images_per_tfrecord=1000):
    record_count = 0
    image_count = 0

    tfrecord_path = tfrecord_dir / f"data_{record_count}.tfrecord"
    writer = tf.io.TFRecordWriter(str(tfrecord_path))

    for images, labels in data_flow:
        for image, label in zip(images, labels):
            if image_count >= images_per_tfrecord:
                writer.close()
                record_count += 1
                image_count = 0
                tfrecord_path = tfrecord_dir / f"data_{record_count}.tfrecord"
                writer = tf.io.TFRecordWriter(str(tfrecord_path))

            tf_example = serialize_example(image, label)
            writer.write(tf_example)
            image_count += 1

    writer.close()
    print(f"Written {record_count + 1} TFRecord files.")


In [11]:
# Charger les données en utilisant des générateurs et les stocker temporairement sur le disque
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
data_flow = data_generator.flow_from_directory(NEW_TRAIN, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)

# Stocker les données sous forme de fichiers TFRecord temporaires pour éviter de surcharger la mémoire
tfrecord_dir = CWD / "tfrecord_temp"
tfrecord_dir.mkdir(parents=True, exist_ok=True)

write_tfrecord(data_flow, tfrecord_dir)

Found 12485 images belonging to 3 classes.


KeyboardInterrupt: 

In [ ]:
def parse_tfrecord_fn(example):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.decode_jpeg(example['image'], channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    label = tf.io.parse_tensor(example['label'], out_type=tf.float32)
    return image, label


In [ ]:
def load_dataset(tfrecord_dir):
    tfrecord_files = [str(file) for file in tfrecord_dir.glob('*.tfrecord')]
    raw_dataset = tf.data.TFRecordDataset(tfrecord_files)
    dataset = raw_dataset.map(parse_tfrecord_fn)
    return dataset

full_dataset = load_dataset(tfrecord_dir)
full_dataset = full_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
# Utiliser la validation croisée avec KFold
kf = KFold(n_splits=K, shuffle=True, random_state=42)
fold_no = 1

all_images = []
all_labels = []

for images, labels in full_dataset:
    all_images.append(images.numpy())
    all_labels.append(labels.numpy())

all_images = np.concatenate(all_images)
all_labels = np.concatenate(all_labels)

for train_index, val_index in kf.split(all_images):
    print(f'Training on fold {fold_no}')
    K.clear_session()
    
    new_model, base_model = create_model()
    for layer in base_model.layers:
        layer.trainable = False

    new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                      loss='categorical_crossentropy',
                      metrics=['accuracy', tf.keras.metrics.Recall()])

    es = EarlyStopping(monitor="val_accuracy", mode="max", patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)
    
    train_data = tf.data.Dataset.from_tensor_slices((all_images[train_index], all_labels[train_index]))
    val_data = tf.data.Dataset.from_tensor_slices((all_images[val_index], all_labels[val_index]))

    train_data = train_data.batch(BATCH_SIZE).map(lambda x, y: (preprocess_input(x), y))
    val_data = val_data.batch(BATCH_SIZE).map(lambda x, y: (preprocess_input(x), y))

    history = new_model.fit(train_data,
                            validation_data=val_data,
                            epochs=EPOCHS,
                            callbacks=[es, reduce_lr])

    # Fine-tuning progressif
    for num_layers in [5, 10, len(base_model.layers)]:
        for layer in base_model.layers[-num_layers:]:
            layer.trainable = True
        
        new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                          loss='categorical_crossentropy',
                          metrics=['accuracy', tf.keras.metrics.Recall()])

        history_fine_tune = new_model.fit(train_data,
                                          validation_data=val_data,
                                          epochs=EPOCHS,
                                          callbacks=[es, reduce_lr])
    
    scores = new_model.evaluate(val_data)
    print(f'Score for fold {fold_no}: {new_model.metrics_names[1]} of {scores[1]}; {new_model.metrics_names[2]} of {scores[2]}')
    fold_no += 1

Training on fold 1


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Entraînement final sur l'ensemble des données
train_dataset = preprocess_dataset(image_dataset_from_directory(
    directory=NEW_TRAIN,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True))

validation_dataset = preprocess_dataset(image_dataset_from_directory(
    directory=NEW_VAL,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True))

test_dataset = preprocess_dataset(image_dataset_from_directory(
    directory=NEW_TEST,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False))

In [ ]:
new_model, base_model = create_model()
for layer in base_model.layers:
    layer.trainable = False

new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Recall()])

history = new_model.fit(train_dataset,
                        validation_data=validation_dataset,
                        epochs=EPOCHS,
                        callbacks=[es, reduce_lr])

test_loss, test_accuracy, test_recall = new_model.evaluate(test_dataset)
print(f"Loss on test dataset: {test_loss}")
print(f"Accuracy on test dataset: {test_accuracy}")
print(f"Recall on test dataset: {test_recall}")

In [ ]:
new_model.save('my_new_model_finetuned_remastered.keras')

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Prédire les classes sur le jeu de test
test_labels = np.concatenate([y for x, y in test_dataset], axis=0)
predictions = new_model.predict(test_dataset)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

# Calculer la matrice de confusion
cm = confusion_matrix(true_labels, predicted_labels)

# Tracer la heatmap de la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names.values(), yticklabels=class_names.values())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()